In [ ]:
# import das libs necessárias
import pandas as pd # trabalhar com dataframes
import numpy as np # realizacao de algumas operacoes com matrizes

#imagens
import cv2 # transformacoes faceis em imagens
from PIL import Image # trabalhar com imagens

# ferramentas
import glob # exploracao de diretorios
from pylab import *
import tqdm
import importlib
import time

# plot 
import matplotlib.pyplot as plt # plotagem
%matplotlib inline

# Machine Learning
from sklearn.neural_network import MLPClassifier 
from sklearn.model_selection import GridSearchCV # Grid Search para rodar todos os parametros

#importando ferramentas já implementadas anteriormente
import sampling #retorna uma amostra para treino
importlib.reload(sampling)
import PCA_and_Inverse
importlib.reload(PCA_and_Inverse)
import feature_selection
importlib.reload(feature_selection)

## Cria uma amostra das imagens do tamanho definido e carrega em dataframes

In [ ]:
#carregando amostra dos dados
X, Y = sampling.get_sample(400)

## Normaliza os dados

In [ ]:
X = sampling.get_min_max_normalization(data=X)

## Extração de características

- Calcula as Componentes principais para os dados carregados

In [ ]:
pca = PCA_and_Inverse.PCA_and_Inverse()
pca_components = pca.get_PCA(X=X, n_components=64)

## Seleção de características

- Calcula as melhores features a serem utilizadas pelos métodos de:
    - Informação Mútua
    - Algoritmo Genético

#### Informação Mútua

In [ ]:
mic_threshold = 0.05
mic_features_df = feature_selection.get_MIC_features(X=X, Y=Y)
mic_features = list(mic_features_df[mic_features_df[0] > mic_threshold].index)

#### Algoritmo Genético

In [ ]:
ga_features = feature_selection.get_GA_features(X=X, Y=Y)
ga_features

## Gera as combinações de parâmetros para rodar a rede neural

In [ ]:
mlp_classifier = MLPClassifier(solver="sgd", learning_rate="constant")

parameters = [{"hidden_layer_sizes":[(10,), (25,), #variando o tamanho de uma única camada escondida
                                     (10, 15), # duas camadas ocultas com 10 e 15 neuronios cada
                                     (10, 10, 10)], # três camadas ocultas com 10 neuronios cada
               "learning_rate_init":[0.001, 0.01, 0.1, 0.5, 0.9],
               "activation":['logistic', 'relu']}]
scoring = ["accuracy", "precision", "recall", "roc_auc"]

result_column_list= ['param_activation',
                     'param_hidden_layer_sizes',
                     'param_learning_rate_init',
                     'mean_test_accuracy',
                     'mean_test_precision',
                     'mean_test_recall',
                     'mean_test_roc_auc']

#### Roda todas as combinações da rede neural para todas as features

In [ ]:
grid_search_full = GridSearchCV(mlp_classifier, parameters, verbose=5, n_jobs=-1, cv=3, scoring=scoring, refit=False)

In [ ]:
grid_search_full.fit(X, Y)

In [ ]:
full_result = pd.DataFrame(grid_search_full.cv_results_)[result_column_list]
full_result.to_csv("Sample_400_Full.csv")

#### Roda todas as combinações da rede neural para as features extraídas pelo PCA

In [ ]:
grid_search_pca = GridSearchCV(mlp_classifier, parameters, verbose=5, n_jobs=-1, cv=3, scoring=scoring, refit=False)

In [ ]:
grid_search_pca.fit(pca_components, Y)

In [ ]:
pd.DataFrame(grid_search_pca.cv_results_)[]

In [ ]:
pca_result = pd.DataFrame(grid_search_pca.cv_results_)[result_column_list]
pca_result.to_csv("Sample_400_PCA.csv")

#### Roda todas as combinações da rede neural para as features selecionadas pela Informação Mútua (MIC)

In [ ]:
grid_search_mic = GridSearchCV(mlp_classifier, parameters, verbose=5, n_jobs=-1, cv=3)

In [ ]:
grid_search_mic.fit(X[mic_features], Y)

In [ ]:
mic_result = pd.DataFrame(grid_search_mic.cv_results_)[result_column_list]
mic_result.to_csv("Sample_400_MIC.csv")

#### Roda todas as combinações da rede neural para as features selecionadas pelo Algoritmo Genético (GA)

In [ ]:
grid_search_ga = GridSearchCV(mlp_classifier, parameters, verbose=5, n_jobs=-1, cv=3)

In [ ]:
grid_search_ga.fit(pca_components, Y)

In [ ]:
ga_result = pd.DataFrame(grid_search_ga.cv_results_)[result_column_list]
ga_result.to_csv("Sample_400_GA.csv")